In [ ]:
%load_ext autoreload
%autoreload 2

# Interacting with Vault
For convinience, this app has been configured to be used with [Vault](https://www.vaultproject.io).  With Vault your app is provided a token that allows it to access secrets:
- db usernames/passwords
- other credentials
Over a secure API rather than having to store these items directly with each instance of your app.  This makes it MUCH easier to move your deployments around or scale them out by creating replicas.

Here, we experiment with the best ways to interact with Vault and keep tokens up to date.  This starts with an understanding of the different toke types available in Vault.  [Token Overview](https://www.vaultproject.io/docs/concepts/tokens)

In [ ]:
import hvac
from piesafe import piesafe
from instance import config
from pprint import pprint

piesafe.init_logging(__name__)

vault token create -orphan=True -no-default-policy=True -policy=omop-policy
Key                  Value
---                  -----
token                s.a90uT7eCWr5UKIg1IM8cqjJC
token_accessor       eC8ARgmue6pdn8kqBtnVBuxg
token_duration       768h
token_renewable      true
token_policies       ["omop-policy"]
identity_policies    []
policies             ["omop-policy"]


➜  ~ vault token create
Key                  Value
---                  -----
token                s.OWmIyZCL86xohmh585XsBxtb
token_accessor       PflejoHNGrSGxGHps66GSDAz
token_duration       768h
token_renewable      true
token_policies       ["default"]
identity_policies    ["vault_admin"]
policies             ["default" "vault_admin"]
➜  ~ 


In [ ]:
named_token='s.'
orph_token='s.'
vtc_tok='s.'
pv_tok = 's.'


named_vault = hvac.Client(url=config.VAULT_SERVER,
                   token=named_token)

orph_vault = hvac.Client(url=config.VAULT_SERVER,
                   token=orph_token)

vtc_vault = hvac.Client(url=config.VAULT_SERVER,
                   token=vtc_tok)

pv_vault = hvac.Client(url=config.VAULT_SERVER,
                   token=pv_tok)

In [ ]:
nt = named_vault.lookup_token()
ot = orph_vault.lookup_token()
vt = vtc_vault.lookup_token()
pv = pv_vault.lookup_token()
print('Named Token \n')
pprint(nt)
print(nt.keys())
print(nt.get('data').keys())
print()

print("Orphan token \n")
pprint(ot)
print(ot.keys())
print(ot.get('data').keys())
print()

print('Vault token create \n')
pprint(vt)
print(vt.keys())
print(vt.get('data').keys())
print()

print("Pieval token \n")
pprint(pv)
print(pv.keys())
print(pv.get('data').keys())
print()

In [ ]:
pv

In [ ]:
pv_vault.renew_token()

In [ ]:
pv_vault.lookup_token()

# Approle auth
pieval_role



In [ ]:
ar = hvac.Client(url=config.VAULT_SERVER)
ar.auth_approle('','')

In [ ]:
ar.lookup_token()

In [ ]:
ar.read('cdi3/db/cdi3sql01/prod')

# Using Piesafe

In [ ]:
vault_client = piesafe.init_vault(None,
                   vault_server=config.VAULT_SERVER,
                   vault_role_id='',
                   vault_secret_id='')
pprint(vault_client.lookup_token())
print()
print()
vault_client.read('cdi3/db/cdi3sql01/dev')

In [ ]:
vault_client = piesafe.init_vault(None,
                   vault_server=config.VAULT_SERVER,
                   vault_token=pv_tok)
vault_client.read('cdi3/db/cdi3sql01/dev')